Import

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, SimpleRNN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Model
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


In [2]:
# import pandas as pd

# # Load the CSV file
# input_file = 's_aes_Quadruplets.csv'  # Replace with your input file name
# df = pd.read_csv(input_file)

# # Select the first 1000 records
# df_first_10000 = df.head(3000000)

# # Save the selected records to another CSV file
# output_file = 'output_first_3000000.csv'  # Replace with your desired output file name
# df_first_10000.to_csv(output_file, index=False)

# print(f"First 1000 records saved to {output_file}")


In [2]:
print(tf.config.list_physical_devices('GPU'))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Upload the data

In [3]:
data = pd.read_csv('output_first_5000000.csv')

In [4]:
y = data[['key']]
y = y.iloc[::5].reset_index(drop=True)
y.head()

,key
0,25c2
1,af42
2,da99
3,5be9
4,691a


Data prepration

In [5]:
x = data.drop(['key', 'before_xor','iv'], axis=1)
print(len(x))
x=x.values.reshape(-1, 10)
# # Display the result
x = pd.DataFrame(x, columns=['plaintext1', 'ciphertext1','plaintext2',  'ciphertext2','plaintext3',  'ciphertext3','plaintext4','ciphertext4','plaintext5',  'ciphertext5'])
x.head()

5000000


,plaintext1,ciphertext1,plaintext2,ciphertext2,plaintext3,ciphertext3,plaintext4,ciphertext4,plaintext5,ciphertext5
0,8d54,a26b,8282,3db3,b87c,874e,b232,d81a,afb1,659d
1,bcce,c524,c8ba,a156,2dbe,d451,9841,21a2,c636,2fdd
2,8912,592c,30f8,70ce,6ea8,3e94,3efd,becc,4db9,2b2e
3,f48a,fd42,e149,a884,5fc7,e60c,fcae,0561,cc0f,15c8
4,df05,e8d4,85b1,726b,22dd,c500,8750,3082,c98a,4e5c


In [6]:
z=data[['iv']]
z_filtered = z.iloc[::5].reset_index(drop=True)
z_filtered.head()

,iv
0,a56d
1,c695
2,20bc
3,2218
4,2c10


In [7]:
x['iv'] = z_filtered
print(len(x))

1000000


In [8]:

# Convert all hexadecimal entries to binary representation in the entire dataframe



def convert_to_binary(hex_number):
    return bin(int(hex_number, 16))[2:].zfill(16)

x = x.applymap(convert_to_binary)
y = y.applymap(convert_to_binary)





X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

plaintext_train1 = np.array([list(map(int, list(row))) for row in X_train['plaintext1'].values])
ciphertext_train1 = np.array([list(map(int, list(row))) for row in X_train['ciphertext1'].values])
plaintext_train2 = np.array([list(map(int, list(row))) for row in X_train['plaintext2'].values])
ciphertext_train2 = np.array([list(map(int, list(row))) for row in X_train['ciphertext2'].values])
plaintext_train3 = np.array([list(map(int, list(row))) for row in X_train['plaintext3'].values])
ciphertext_train3 = np.array([list(map(int, list(row))) for row in X_train['ciphertext3'].values])
plaintext_train4 = np.array([list(map(int, list(row))) for row in X_train['plaintext4'].values])
ciphertext_train4 = np.array([list(map(int, list(row))) for row in X_train['ciphertext4'].values])
plaintext_train5 = np.array([list(map(int, list(row))) for row in X_train['plaintext5'].values])
ciphertext_train5 = np.array([list(map(int, list(row))) for row in X_train['ciphertext5'].values])

plaintext_test1 = np.array([list(map(int, list(row))) for row in X_test['plaintext1'].values])
ciphertext_test1 = np.array([list(map(int, list(row))) for row in X_test['ciphertext1'].values])
plaintext_test2 = np.array([list(map(int, list(row))) for row in X_test['plaintext2'].values])
ciphertext_test2 = np.array([list(map(int, list(row))) for row in X_test['ciphertext2'].values])
plaintext_test3 = np.array([list(map(int, list(row))) for row in X_test['plaintext3'].values])
ciphertext_test3 = np.array([list(map(int, list(row))) for row in X_test['ciphertext3'].values])
plaintext_test4 = np.array([list(map(int, list(row))) for row in X_test['plaintext4'].values])
ciphertext_test4 = np.array([list(map(int, list(row))) for row in X_test['ciphertext4'].values])
plaintext_test5 = np.array([list(map(int, list(row))) for row in X_test['plaintext5'].values])
ciphertext_test5 = np.array([list(map(int, list(row))) for row in X_test['ciphertext5'].values])

iv_train = np.array([list(map(int, list(row))) for row in X_train['iv'].values])
iv_test = np.array([list(map(int, list(row))) for row in X_test['iv'].values])
key_train = np.array([list(map(int, list(row))) for row in y_train['key'].values])
key_test = np.array([list(map(int, list(row))) for row in y_test['key'].values])
# Stack the arrays to create the transformed datasets
X_train_transformed = np.stack([plaintext_train1, ciphertext_train1,plaintext_train2,ciphertext_train2,plaintext_train3,ciphertext_train3,iv_train], axis=-1)
X_test_transformed = np.stack([plaintext_test1, ciphertext_test1,plaintext_test2,ciphertext_test2,plaintext_test3,ciphertext_test3,iv_test], axis=-1)

# For the y_train and y_test
# concat_train = np.concatenate((key_train, iv_train), axis=1)
# concat_test = np.concatenate((key_test, iv_test), axis=1)
y_train_transformed = key_train[..., np.newaxis]
y_test_transformed = key_test[..., np.newaxis]
# arr = X_train.apply(lambda row: [[int(row['plaintext'][i]), int(row['ciphertext'][i]), int(row['iv'][i])] for i in range(16)], axis=1).to_numpy()

# X_train_transformed = np.zeros((X_train.shape[0], 16, 3), dtype=int)
# for i in range(X_train.shape[0]):
#     for j in range(16):
#         X_train_transformed[i, j, :] = arr[i][j]

# arr = X_test.apply(lambda row: [[int(row['plaintext'][i]), int(row['ciphertext'][i]), int(row['iv'][i])] for i in range(16)], axis=1).to_numpy()

# X_test_transformed = np.zeros((X_test.shape[0], 16, 3), dtype=int)
# for i in range(X_test.shape[0]):
#     for j in range(16):
#         X_test_transformed[i, j, :] = arr[i][j]

# arr = y_train.apply(lambda row: [[int(row['key'][i])] for i in range(16)], axis=1).to_numpy()

# y_train_transformed = np.zeros((y_train.shape[0], 16,1 ), dtype=int)
# for i in range(y_train.shape[0]):
#     for j in range(16):
#         y_train_transformed[i, j, :] = arr[i][j]



# y_test_transformed = np.zeros((y_test.shape[0], 16,1 ), dtype=int)
# for i in range(y_test.shape[0]):
#     for j in range(16):
#         y_test_transformed[i, j, :] = arr[i][j]







In [10]:
y_test.shape

(60000, 1)

Build the model

In [11]:
# #BidirectionalRNN
# model = keras.Sequential()

# # Add a Dense layer
# model.add(layers.Dense(128, activation='relu',input_shape=(X_train_transformed.shape[1], X_train_transformed.shape[2])))
# model.add(layers.BatchNormalization())

# # Convert SimpleRNN to Bidirectional RNN
# model.add(layers.Bidirectional(layers.SimpleRNN(800), input_shape=(X_train_transformed.shape[1], 128)))

# # Add a Dense layer
# model.add(layers.Dense(1024, activation='relu'))
# model.add(layers.BatchNormalization())

# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.BatchNormalization())

# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.BatchNormalization())


# # Add a Dense layer with Sigmoid activation
# model.add(layers.Dense(16, activation='sigmoid'))

# # Compile the model
# reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, min_lr=1e-15, verbose=1)


# # Compile the model
# optimizer = keras.optimizers.Adam(learning_rate=1e-4,clipnorm=1.0)
# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy']) # Changed accuracy metric to binary_accuracy
# # Train the model
# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16, 128)           1024      
                                                                 
 batch_normalization (BatchN  (None, 16, 128)          512       
 ormalization)                                                   
                                                                 
 bidirectional (Bidirectiona  (None, 1600)             1486400   
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1024)              1639424   
                                                                 
 batch_normalization_1 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                        

In [11]:
model=keras.models.load_model('maybe2.h5')
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, min_lr=1e-15, verbose=1)


# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=1e-4,clipnorm=1.0)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16, 128)           1024      
                                                                 
 batch_normalization (BatchN  (None, 16, 128)          512       
 ormalization)                                                   
                                                                 
 bidirectional (Bidirectiona  (None, 1600)             1486400   
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1024)              1639424   
                                                                 
 batch_normalization_1 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                        

In [16]:
with tf.device('/GPU:0'):
    model.fit(X_train_transformed, y_train_transformed, epochs=200, batch_size=10000, callbacks=[reduce_lr], verbose=1)


Epoch 1/200
90/90 [==============================] - 57s 631ms/step - loss: 0.4872 - binary_accuracy: 0.7570 - lr: 1.0000e-04
Epoch 2/200
90/90 [==============================] - 57s 632ms/step - loss: 0.4859 - binary_accuracy: 0.7581 - lr: 1.0000e-04
Epoch 3/200
90/90 [==============================] - 57s 632ms/step - loss: 0.4851 - binary_accuracy: 0.7585 - lr: 1.0000e-04
Epoch 4/200
90/90 [==============================] - 57s 634ms/step - loss: 0.4842 - binary_accuracy: 0.7591 - lr: 1.0000e-04
Epoch 5/200
90/90 [==============================] - 57s 634ms/step - loss: 0.4835 - binary_accuracy: 0.7595 - lr: 1.0000e-04
Epoch 6/200
90/90 [==============================] - 57s 634ms/step - loss: 0.4825 - binary_accuracy: 0.7602 - lr: 1.0000e-04
Epoch 7/200
90/90 [==============================] - 57s 634ms/step - loss: 0.4817 - binary_accuracy: 0.7607 - lr: 1.0000e-04
Epoch 8/200
90/90 [==============================] - 57s 634ms/step - loss: 0.4807 - binary_accuracy: 0.7614 - lr: 1.0

Evaluate the model

In [17]:


y_pred = (np.array(model.predict(X_test_transformed)) >= 0.5).astype("int32")




3125/3125 [==============================] - 20s 6ms/step


In [18]:
y_test_transformed=y_test_transformed.squeeze()
y_test_transformed.shape

(100000, 16)

In [19]:
print(y_pred[2345])
print(y_test_transformed[2345])

[1 1 1 1 1 0 1 1 1 1 1 0 0 0 1 1]
[0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0]


In [20]:
correct_count = 0

# Iterate over each array
for i in range(y_test_transformed.shape[0]):
    if np.array_equal(y_test_transformed[i], y_pred[i]):
        correct_count += 1

# Calculate accuracy
total_arrays = y_test_transformed.shape[0]
accuracy = correct_count / total_arrays

print(f"Number of correct predictions: {correct_count}")
print(f"Accuracy: {accuracy * 100:.2f}%")

Number of correct predictions: 2
Accuracy: 0.00%


In [21]:
y_pred = (np.array(model.predict(X_train_transformed)) >= 0.5).astype("int32")


28125/28125 [==============================] - 173s 6ms/step


In [22]:
y_train_transformed=y_train_transformed.squeeze()

In [23]:
correct_count = 0

# Iterate over each array
for i in range(y_train_transformed.shape[0]):
    if np.array_equal(y_train_transformed[i], y_pred[i]):
        correct_count += 1

# Calculate accuracy
total_arrays = y_train_transformed.shape[0]
accuracy = correct_count / total_arrays

print(f"Number of correct predictions: {correct_count}")
print(f"Accuracy: {accuracy * 100:.2f}%")

Number of correct predictions: 110919
Accuracy: 12.32%


In [24]:
print(y_train_transformed[0])
print(y_pred[0])

[0 0 0 1 1 1 0 1 0 1 1 1 1 1 1 1]
[0 0 1 1 1 0 0 0 0 1 1 1 1 1 1 1]


In [25]:
model.save('maybe2.h5')